In [13]:
import zipline
from zipline.api import order_target_percent, symbol
from zipline.api import set_commission, set_slippage
from zipline.api import schedule_function, date_rules, time_rules 

from zipline.finance.commission import PerDollar
from zipline.finance.slippage import VolumeShareSlippage, FixedSlippage

from datetime import datetime
import pytz
import matplotlib.pyplot as plt
import pyfolio as pf
import pandas as pd
import numpy as np
from scipy import stats

%matplotlib inline

In [14]:
# Model Settings
initial_portfolio = 100000
momentum_window = 125
minimum_momentum = 40
portfolio_size = 30
vola_window = 20

# Commission and Slippage Settings
enable_commission = True
commission_pct = 0.001
enable_slippage = True
slippage_volume_limit = 0.025
slippage_impact = 0.05

In [15]:
# Momementum scure calculations
def momentum_score(ts):
    """
    Input: Price time series.
    Output: Annualized exponential regression slope multiplied by its corresponding R^2
    """
    # Make a list of consecutive numbers
    x = np.arange(len(ts))
    # Get logs of prices in the time series
    log_ts = np.log(ts)
    # Calculate regression values
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, log_ts)
    # Annualized percent
    annualized_slope = (np.power(np.exp(slope), 252) - 1) * 100
    # Adjust for fitness
    score = annualize_slope * (r_value ** 2)
    
    return score

In [16]:
# Volatility is used for position sizing 
def volatility(ts):
    return ts.pct_change().rolling(vola_window).std().iloc[-1]

In [17]:
# Percentage return for the previous month
def output_progress(context):
    """
    Output some performance numbers during backtest run.
    This code just prints out teh past month's performance
    so that one has something to look at while the backtest runs
    """
    
    # Get today's date
    today = zipline.api.get_datetime().date()
    
    # Calculate percent difference since last month
    perf_pct = (context.portfolio.portfolio_value / context.last_month) - 1
    
    # Print performance, format as percent with two decimals
    print("{} - Last month results: {:.2%}".format(today, perf_pct))
    
    # Remember today's portfolio value for the next month's calculation 
    context.last_month = context.portfolio.portfolio_value
    

In [18]:
# Startup routine assumes that one has a local file with historical index membership
def initialize(context):
    
    # Set zipline benchmark to false
    context.set_benchmark(False)
    
    
    # Set commission and slipage
    if enable_commission:
        comm_model = PerDollar(cost=commission_pct)
    else:
        comm_model = PerDollar(cost=0.0)
    set_commission(comm_model)
    
    if enable_slippage:
        slippage_model = VolumeShareSlippage(volume_limit=slippage_volume_limit, price_impact=slippage_impact)
    else:
        slippage_model = FixedSlippage(spread=0.0)
    set_slippage(slappage_model)
    
    # Functionality below is used only for progress output
    context.last_month = initial_portfolio
    
    # Store index membership
    context.index_memebers = pd.read_csv('sp500_membership.csv', index_col=0, parse_dates=[0])
    
    #Schedule rebalance monthly
    schedule_function(
        func=rebalance,
        date_rule=date_rules.month_start(),
        time_rule=time_rules.market_open()
    )

In [20]:
def rebalance(context, data):
    # Write some progress output during the backtest
    output_progress(context)
    
    # Find out what stocks were memebers of the index on a specific date (todays date)
    today = zipline.api.get_datetime()
    
    # Second, get the index makeup for all days prior to a specific date (todays date)
    all_prior = context.index_memebers.loc[context.index_members.index < today]
    
    # Now let's snag teh first column of the last, i.e. latest entry
    latest_day = all_prior.iloc[1, 0]
    
    # Split the text string with ticker into a list
    list_of_tickers = latest_day.split(',')
    
    # Finally, get the zipline sybmols for the tickers
    todays_universe = [symbol(ticker) for ticker in list_of_tickers]
    
    # Get historical data
    hist = data.history(todays_universe, "close", momentum_window, "1d")
    
    # Make momentum ranking table
    ranking_table = hist.apply(momentum_score).sort_values(ascending=False)
    
    """
    Sell logic:
    First, we check if any existing positions should be sold.
    - Sell if stock is no longer part of index
    - Sell if stock has lower momentum value
    """
    kept_positions = list(context.portfolio.positions.keys())
    for security in context.portfolio.positions:
        if (security not in todays_universe):
            order_target_percent(security, 0.0)
        elif ranking_table[security] < minimum_momentum:
            order_target_percent(security, 0.0)
            kept_positions.remove(security)
            
    """
    Stock selection logic:
    Check how many stocks we are keeping for last month. Fill from top of ranking list, 
    until we reach the desired total number of portfolio holdings.
    """
    replacement_stocks = portfolio_size - len(kept_positions)
    buy_list = ranking_table.loc[
        ~ranking_table.index.isin(kept_positions)][:replacement_stocks]
    
    new_portfolio = pd.concat((buy_list, ranking_table.loc[ranking_table.index.isin(kept_positions)])
                              
    #Calculate inverse volatility for stocks, and make target position weights                    
    vola_table = hist[new_portfolio.index].apply(volatility)
    inv_vola_table = 1 / vola_table
    sum_inv_vola = np.sum(inv_vola_table)
    vola_target_weights = inv_vola_table / sum_inv_vola
                              
    for security, rank in new_portfolio.iteritems():
        weight = vola_target_weights[security]
        if security in kept_positions:
            order_target_percent(security, weight)                      
        else:
            if ranking_table[security] > minimum_momentum:
                order_target_percent(security, weight)

SyntaxError: invalid syntax (<ipython-input-20-c8dc2cfeb0b8>, line 52)

In [21]:
def analyze(context, perf):
    perf['max'] = perf.portfolio_value.cummax()
    perf['dd'] = (perf.portfolio_value / perf['max']) - 1
    maxdd = perf['dd'].min()
    ann_ret = (np.power((perf.portfolio_value.iloc[-1] / perf.portfolio_value.iloc[0]), (252 / len(perf))) - 1)
               
    print("Annualized return: {:..2%} Max drawdown: {:.2%}".format(ann_ret, maxdd)) 
    return 

In [22]:
start = datetime(1997, 1, 1, 8, 15, 12, 0, pytz.UTC)
end = datetime(2018, 12, 31, 8, 15, 12, 0, pytz.UTC)
perf = zipline.run_algorithm(
    start=start, end=end, 
    initialize=initialize,
    analyze=analyze,
    capital_base=initial_portfolio,
    data_frequency= 'daily',
    bundle='random_stock_data'

)

AssertionError: 